In [33]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('../input/digit-recognizer/train.csv')

In [34]:
data = np.array(data)
m, n =data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T  # splitting the data into training and test data
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev/255.   # DID NOT UNDERSTAND THE PURPOSE maybe this gives a fraction which tells the brightness of the pixel

data_train = data[1000:m].T 
Y_train = data_train[0]   # contains the labels of the 41000 (m-1000) eg
X_train = data_train[1:n]
X_train = X_train/255.
_,m_train = X_train.shape  # total no of eg in the set

In [35]:
# FUNCTIONS

def init_params():  #generating the params randomly
    W1 = np.random.rand(16, 784) - 0.5  # 10 x 784 mtx of 1st layer params. 10 is the no of neurons and 784 is the no of inputs to each neuron
    #b1 = np.random.rand(16,1) - 0.5   #  1st arg tells no fo neurons and 2nd tells no of inputs to each neuron
    b1 = np.random.rand(16, 1) - 0.5
    W2 = np.random.rand(10, 16) - 0.5 
    b2 = np.random.rand(10,1) - 0.5  
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1   # Z1 is a 10 x m mtx. b1 adjusts to the dim of w1.X
    A1 = ReLU(Z1)     # same dim as Z1
    Z2 = W2.dot(A1) + b2    # dim is 10 x m
    A2 = softmax(Z2)    # same dim as Z2
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0    # produces array in which each ele is 0 or 1. if ele in Z is positive no then 1 if not then 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))   # zero mtx of dim 41000(y size) x 10 (9(y max) + 1)
    one_hot_Y[np.arange(Y.size), Y] = 1    # 1st arg: array of 0 to 41000 and 2nd arg: array of labels
    # each ele of the 1st arg corresponds to the respective ele of the 2nd arg. This becomes the row no and col no arg for one_hot_y
    # In this sense one_hot_y contains the right answers of each eg.
    one_hot_Y = one_hot_Y.T   # dim 10 x m (41000)
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y   # gives the error for each of the eg. dim is 10 x m (same as A2)
    dW2 = 1 / m * dZ2.dot(A1.T)   # dim 10 x 10. Equal to the dvt of the cost wrt W2 (params of 2nd layer(output layer))
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)    # dim 10 x m. Using * corresponding eles are multiplied individually
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, a):
    W1 = W1 - a * dW1
    b1 = b1 - a * db1
    W2 = W2 - a * dW2
    b2 = b2 - a * db2
    return W1, b1, W2, b2


In [36]:
def get_prediction(A2):
    return np.argmax(A2, 0)    # give the index of the output which has the max magnitude. It is equal to the digit (predicted output)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, a, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_train)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X_train, Y_train)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, a)
        if i%10 == 0:
            print("iteration: ", i)
            predictions = get_prediction(A2)
            print("shape of pred: ", predictions.shape)
            print(get_accuracy(predictions, Y_train))
    return W1, b1, W2, b2        

In [37]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

iteration:  0
shape of pred:  (41000,)
[1 2 1 ... 1 9 1] [7 0 3 ... 7 4 7]
0.09402439024390244
iteration:  10
shape of pred:  (41000,)
[7 2 7 ... 2 4 6] [7 0 3 ... 7 4 7]
0.21314634146341463
iteration:  20
shape of pred:  (41000,)
[7 4 7 ... 2 4 3] [7 0 3 ... 7 4 7]
0.29797560975609755
iteration:  30
shape of pred:  (41000,)
[7 4 7 ... 9 4 7] [7 0 3 ... 7 4 7]
0.35873170731707316
iteration:  40
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.42204878048780486
iteration:  50
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.5003658536585366
iteration:  60
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.5639024390243902
iteration:  70
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.6033170731707317
iteration:  80
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.636
iteration:  90
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] [7 0 3 ... 7 4 7]
0.6650731707317074
iteration:  100
shape of pred:  (41000,)
[7 3 3 ... 9 4 7] 

In [38]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_prediction(A2)
    return predictions

In [39]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
print(get_accuracy(dev_predictions, Y_dev))
print(dev_predictions.shape)

[1 9 8 2 3 4 6 6 6 6 3 1 3 1 6 8 3 9 1 1 8 8 8 2 8 1 4 6 1 2 8 5 4 2 6 9 2
 5 1 1 8 4 2 6 5 2 8 9 4 3 8 7 0 0 3 8 0 1 8 5 9 8 7 8 3 4 1 8 6 4 1 7 7 5
 9 0 6 1 1 4 2 7 3 2 3 4 3 2 1 4 1 0 1 4 4 4 8 7 4 8 7 2 1 9 8 8 0 0 1 6 3
 3 9 8 6 9 1 3 4 1 4 2 5 3 9 0 2 5 6 7 9 9 9 8 6 7 8 4 6 7 6 8 0 1 5 7 4 8
 3 3 3 1 4 3 3 1 7 8 7 4 0 7 9 2 4 7 4 7 1 4 6 3 9 9 1 1 0 9 1 9 7 1 1 1 0
 2 3 2 8 7 4 1 1 2 9 5 0 8 7 7 9 5 5 0 9 7 6 2 3 8 6 5 0 0 3 7 9 6 3 5 3 0
 5 6 8 7 3 5 7 2 4 6 5 6 0 7 9 7 0 2 2 5 8 5 4 5 6 7 3 6 2 4 7 1 2 5 0 1 8
 4 7 4 4 0 4 5 6 1 7 1 6 5 9 1 5 3 0 6 4 9 6 3 3 2 6 1 6 5 5 6 1 4 0 4 2 0
 7 7 6 7 1 3 7 6 4 3 0 1 9 9 4 9 9 5 9 2 7 3 4 1 5 0 0 3 6 4 1 9 4 3 1 5 2
 5 1 9 6 1 3 1 1 5 4 1 3 5 8 2 2 9 4 3 8 6 1 6 5 3 9 9 3 5 5 8 2 8 8 9 6 2
 4 2 4 3 9 2 5 5 9 5 6 6 2 7 2 2 8 8 9 1 8 1 8 7 9 4 5 6 8 9 4 6 7 7 6 8 2
 3 8 4 5 0 1 4 5 1 8 9 2 1 6 4 3 4 7 7 3 9 8 7 2 5 5 9 1 1 1 6 5 1 4 3 7 7
 8 3 8 8 5 1 6 4 3 4 0 6 3 4 1 2 9 7 5 6 8 7 0 5 2 3 8 9 5 8 7 9 1 6 8 5 7
 7 9 8 4 3 1 0 8 3 6 6 5 

In [40]:
W1, b1, W2, b2 = init_params()
print(b1.shape)

(16, 1)
